

---

# 第 1 步｜先看“结果长什么样”：词向量与维度

**你得到的是一个词典（Vocabulary）和一块嵌入矩阵（Embedding Matrix）**：

* 词典大小 $|V|$：保留了 `min_count` 以上频次的词。
* 向量维度 $D$：`vector_size`（比如 300）。
* 每个词对应矩阵中的一行 $\mathbf{v}_w \in \mathbb{R}^D$。

**怎么查看（Gensim 训练版）：**

```python
# view_vectors.py
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format("outputs/w2v_text8_sgns.vec", binary=False)

print("Vocab size |V|:", len(wv.key_to_index))
print("Vector dim D:", wv.vector_size)

for w in ["king","queen","london","computer"]:
    if w in wv:
        vec = wv.get_vector(w)
        print(f"\n{w}: shape={vec.shape}, first10={vec[:10]}")
```

运行 `python view_vectors.py`，你会看到：

* `|V|`（词表大小）
* `D`（向量维度）
* 若干词的向量切片（前 10 维）

> 对应你脚本：`train_text8_gensim.py` 里 `vector_size=300`；保存的是 **输入嵌入（input embedding）**。


In [1]:
# view_vectors.py
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format("outputs/w2v_text8_sgns.vec", binary=False)

print("Vocab size |V|:", len(wv.key_to_index))
print("Vector dim D:", wv.vector_size)

for w in ["king","queen","london","computer"]:
    if w in wv:
        vec = wv.get_vector(w)
        print(f"\n{w}: shape={vec.shape}, first10={vec[:10]}")


Vocab size |V|: 71290
Vector dim D: 300

king: shape=(300,), first10=[-0.31459886  0.25818157 -0.58152425  0.11466006  0.06869234 -0.01890304
 -0.42236814  0.22721554 -0.16372488  0.32195234]

queen: shape=(300,), first10=[-0.13626449  0.3582669  -0.52539974  0.03416476  0.136008   -0.27219403
 -0.10548951  0.268655   -0.11663678  0.29750934]

london: shape=(300,), first10=[ 0.03262839 -0.07578083 -0.30036178  0.04518395  0.06189523 -0.2737656
  0.19897458  0.20811638 -0.22876817  0.34725437]

computer: shape=(300,), first10=[-0.24689974 -0.18490234 -0.46620628 -0.2527385   0.3693195   0.09378664
 -0.11778377 -0.18483984 -0.6295574   0.20734628]



---

# 第 2 步｜数据“先被瘦身”：子采样（Subsampling）

**为什么要子采样？** 高频词（the, of, and）信息量低且频繁，会主导梯度、拖慢训练；子采样按概率丢弃它们。
**公式**（Mikolov）：

$$
p_{\text{drop}}(w)=1-\sqrt{\tfrac{t}{f(w)}}\quad (t\in[10^{-5},10^{-3}])
$$

`sample=1e-3` 就是这个 $t$。

**在 Gensim 哪里设置？**
`Word2Vec(..., sample=1e-3)` —— Gensim 内部自动对子采样生效。

**直观看一下 Text8 上几类高频词会被丢弃的概率：**

```python
# peek_subsampling.py
from collections import Counter
from pathlib import Path
import math

TEXT8 = Path("data/text8")
t = 1e-3  # 你的脚本里就是这个
words = ["the","of","and","king","queen","computer"]
toks = open(TEXT8, "r", encoding="utf-8").read().split()
N = len(toks)
cnt = Counter(toks)

def p_drop(freq, t=t):
    f = freq / N
    return 1.0 - math.sqrt(t / max(f, 1e-12))

for w in words:
    print(f"{w:8s} freq={cnt[w]:8d}  p_drop≈{p_drop(cnt[w]):.3f}")
```

> 你会看到像 `the/of/and` 的 `p_drop` 接近 1，而 “king/queen/computer” 要低得多。


In [2]:
# peek_subsampling.py
from collections import Counter
from pathlib import Path
import math

TEXT8 = Path("data/text8")
t = 1e-3  # 你的脚本里就是这个
words = ["the","of","and","king","queen","computer"]
toks = open(TEXT8, "r", encoding="utf-8").read().split()
N = len(toks)
cnt = Counter(toks)

def p_drop(freq, t=t):
    f = freq / N
    return 1.0 - math.sqrt(t / max(f, 1e-12))

for w in words:
    print(f"{w:8s} freq={cnt[w]:8d}  p_drop≈{p_drop(cnt[w]):.3f}")


the      freq= 1061396  p_drop≈0.873
of       freq=  593677  p_drop≈0.831
and      freq=  416629  p_drop≈0.798
king     freq=    7456  p_drop≈-0.510
queen    freq=    1940  p_drop≈-1.961
computer freq=    5874  p_drop≈-0.701



---

# 第 3 步｜训练样本怎么来的：动态窗口 & 正样本对

**Skip-Gram（跳字模型）**：以中心词 $w$ 预测窗口内的上下文词 $c$。
**动态窗口（Dynamic Window）**：每次训练从 $[1, W]$（`window=5`）均匀采样半径 $b$，只用 $\pm b$ 的邻词作正样本，越近越常被采到。

**在 Gensim 哪里设置？**
`window=5`（动态窗口是其默认策略的一部分）。

**直观看一个位置的“正样本对（w,c）”如何形成：**

```python
# peek_pairs.py
from pathlib import Path
import random
TEXT8 = Path("data/text8")
toks = open(TEXT8, "r", encoding="utf-8").read().split()
i = 123456  # 随便取一个位置
W = 5
b = random.randint(1, W)
L, R = max(0, i-b), min(len(toks), i+b+1)
center = toks[i]
contexts = [toks[j] for j in range(L, R) if j != i]
print("b=", b)
print("center:", center)
print("contexts:", contexts)
```

> 这展示了“给定一次采样下”中心词与它的上下文词们（正样本）。


In [3]:
# peek_pairs.py
from pathlib import Path
import random
TEXT8 = Path("data/text8")
toks = open(TEXT8, "r", encoding="utf-8").read().split()
i = 123456  # 随便取一个位置
W = 5
b = random.randint(1, W)
L, R = max(0, i-b), min(len(toks), i+b+1)
center = toks[i]
contexts = [toks[j] for j in range(L, R) if j != i]
print("b=", b)
print("center:", center)
print("contexts:", contexts)


b= 5
center: no
contexts: ['for', 'trade', 'in', 'manufactured', 'goods', 'tariffs', 'and', 'as', 'a', 'non']



---

# 第 4 步｜负样本从哪儿来：负采样（Negative Sampling）

**思想**：把大规模 softmax 变成多次二分类。对每个正样本 $(w,c)$，再采 $k$ 个词 $n_i$ 当“伪负例”。
**采样分布**：经验最好用 **unigram$^{0.75}$**

$$
P_n(x)\propto f(x)^{0.75}
$$

**在 Gensim 哪里设置？**
`negative=10` 表示每个正样本配 10 个负样本；分布固定为 $0.75$ 幂的 unigram。

**直观看看“被抽为负样本”的高低频倾向：**

```python
# peek_negatives.py
import numpy as np
from collections import Counter
from pathlib import Path
TEXT8 = Path("data/text8")
toks = open(TEXT8, "r", encoding="utf-8").read().split()
cnt = Counter(toks)
vocab = list(cnt.keys())
freq = np.array([cnt[w] for w in vocab], dtype=np.float64)

alpha = 0.75
p = (freq ** alpha) / (freq ** alpha).sum()

# 抽 20 个“负样本候选词”
samples = np.random.choice(vocab, size=20, replace=True, p=p)
print(samples)
```

> 你会看到既有高频词（因为多），也能比“按原频率”更常抽到一些中等频词（因为开了 $0.75$ 次幂，压了最高频）。


In [ ]:
# peek_negatives.py
import numpy as np
from collections import Counter
from pathlib import Path
TEXT8 = Path("data/text8")
toks = open(TEXT8, "r", encoding="utf-8").read().split()
cnt = Counter(toks)
vocab = list(cnt.keys())
freq = np.array([cnt[w] for w in vocab], dtype=np.float64)

alpha = 0.75
p = (freq ** alpha) / (freq ** alpha).sum()

# 抽 20 个“负样本候选词”
samples = np.random.choice(vocab, size=20, replace=True, p=p)
print(samples)

['assembly' 'freedom' 'indeed' 'pooh' 'solitary' 'coastguard' 'goddess'
 'suggestion' 'habitat' 'kennings' 'defense' 'returned' 'decade' 'awlaki'
 'barak' 'bin' 'w' 'antiquities' 'american' 'rickards']


: 


---

# 第 5 步｜目标函数到底优化了什么

每个正样本 $(w,c)$ 的 **SGNS 目标**：

$$
\max \ \underbrace{\log \sigma(\mathbf{u}_c^\top \mathbf{v}_w)}_{\text{正样本打 1}}
\;+\;
\sum_{i=1}^{k}
\underbrace{\log \sigma\!\left(-\mathbf{u}_{n_i}^\top \mathbf{v}_w\right)}_{\text{负样本打 0}}
$$

这等价于最小化它的负值作为 `loss`。其中

* $\mathbf{v}_w$ 是 **输入嵌入（input embedding）**（中心词向量）
* $\mathbf{u}_c$ / $\mathbf{u}_{n_i}$ 是 **输出嵌入（output embedding）**（上下文/负样本向量）
* $\sigma$ 是 Sigmoid

**在 Gensim 里哪里做？**
Gensim 的 C/Python 优化代码内部完成了：

* 取 `window` 内的正样本对
* 采 `negative` 个负样本
* 计算 $\log\sigma(\cdot)$ 与 $\log\sigma(-\cdot)$
* 反向传播、更新两套嵌入矩阵



---

# 第 6 步｜用一个“可见的”小例子演示一次前向 + loss（教学用，PyTorch）

> 这段**不依赖你训练好的模型**，纯教学演示“一个中心词 + 1 个正样本 + 5 个负样本”的数值流。

```python
# sgns_one_step_demo.py
import torch, math
torch.manual_seed(0)

D = 8      # 向量维度（演示用小维度）
K = 5      # 负样本个数

# 随机初始化（真实训练里它们来自 Embedding 矩阵）
v_w    = torch.randn(D)          # 中心词向量 v_w
u_pos  = torch.randn(D)          # 正上下文向量 u_c
u_neg  = torch.randn(K, D)       # K 个负样本向量 u_{n_i}

pos_score = (v_w * u_pos).sum()          # 标量
neg_score = u_neg @ v_w                   # [K]

pos_term = torch.log(torch.sigmoid(pos_score))
neg_term = torch.log(torch.sigmoid(-neg_score)).sum()
loss = -(pos_term + neg_term)

print("pos_score:", pos_score.item())
print("neg_score:", neg_score.tolist())
print("loss:", loss.item())
```

运行看看数值流：

* `pos_score` 越大，`log σ(pos_score)` 越大（更好）；
* `neg_score` 越小（负），`log σ(-neg_score)` 越大（更好）；
* `loss` 是取了负号的和，训练会让它下降。

**这就对应你训练时每个样本在做的事**；只不过真实训练会批量化（mini-batch），并用优化器迭代更新两套嵌入矩阵。


In [1]:
import numpy as np

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sgns_one_step_numpy(D=8, K=5, seed=0):
    rng = np.random.default_rng(seed)
    v_w   = rng.standard_normal(D).astype(np.float32)
    u_pos = rng.standard_normal(D).astype(np.float32)
    u_neg = rng.standard_normal((K, D)).astype(np.float32)

    pos_score = float(v_w @ u_pos)         # 标量
    neg_score = (u_neg @ v_w)              # [K]

    pos_term = np.log(sigmoid(pos_score))
    neg_term = np.log(sigmoid(-neg_score)).sum()
    loss = -(pos_term + neg_term)

    print("pos_score:", pos_score)
    print("neg_score:", neg_score.tolist())
    print("loss:", float(loss))

sgns_one_step_numpy(D=8, K=5, seed=0)


pos_score: -1.4679869413375854
neg_score: [0.374785453081131, -1.6615058183670044, 2.8233561515808105, 4.088198184967041, -0.9111365675926208]
loss: 10.070983284724951



---

# 第 7 步｜参数更新在“哪两块矩阵”上发生

SGNS 同时更新两套参数：

* **输入嵌入矩阵** $\mathbf{V}\in\mathbb{R}^{|V|\times D}$：取第 $w$ 行得到 $\mathbf{v}_w$
* **输出嵌入矩阵** $\mathbf{U}\in\mathbb{R}^{|V|\times D}$：取第 $c$ 行得到 $\mathbf{u}_c$，负样本取 $n_i$ 的行

训练结束后，**常用 $\mathbf{V}$** 作为词向量导出（Gensim 的 `.vec` 就是它）。

> 这就是为什么你能用 `KeyedVectors` 直接查单词向量。

---

# 第 8 步｜Gensim 的若干“隐形细节”

* **学习率（Learning Rate）**：`alpha` 从初始值线性衰减到 `min_alpha`（你没显式改的话用默认）。
* **多线程（workers）**：你设置的 `workers` 个线程并发处理不同句块。
* **动态窗口** 与 **子采样** 在内部自动进行，对你透明。

---

# 第 9 步｜你脚本里的开关与“发生位置”总表

| 训练现象                   | 你脚本里的参数/位置        | 发生在什么时候                        |
| ---------------------- | ----------------- | ------------------------------ |
| 子采样（Subsampling）       | `sample=1e-3`     | 读句子 → 构造训练样本前                  |
| 动态窗口（Dynamic Window）   | `window=5`        | 为每个中心词采 $b$ 决定上下文范围            |
| 负采样（Negative Sampling） | `negative=10`     | 为每个正样本配 10 个负样本                |
| Skip-Gram              | `sg=1`            | 用中心词预测上下文，而不是 CBOW             |
| 嵌入维度                   | `vector_size=300` | 决定 $\mathbf{V},\mathbf{U}$ 的列数 |
| 词表修剪                   | `min_count=5`     | 统计频次后建立词表                      |
| 训练轮数                   | `epochs=5`        | 整个语料反复迭代的次数                    |

---

# 第 10 步｜训练完怎么“验证它学到关系了”

你已经有 `eval_w2v.py`。再加一条\*\*余弦相似度（Cosine Similarity）\*\*查看函数，直观看相近词：

```python
# extra_eval.py
from gensim.models import KeyedVectors
import numpy as np
wv = KeyedVectors.load_word2vec_format("outputs/w2v_text8_sgns.vec", binary=False)

def cos(a,b): return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)+1e-9)

pairs = [("king","queen"), ("paris","france"), ("computer","software")]
for a,b in pairs:
    if a in wv and b in wv:
        print(f"{a:10s} ~ {b:10s} cos={cos(wv[a], wv[b]):.3f}")

print("\nAnalogy: king - man + woman ≈ ?")
if all(w in wv for w in ["king","man","woman"]):
    for w,s in wv.most_similar(positive=["king","woman"], negative=["man"], topn=10):
        print(f"  {w:15s} {s:.4f}")
```


In [ ]:
from gensim.models import KeyedVectors
import numpy as np
wv = KeyedVectors.load_word2vec_format("outputs/w2v_text8_sgns.vec", binary=False)

def cos(a,b): return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)+1e-9)

pairs = [("king","queen"), ("paris","france"), ("computer","software")]
for a,b in pairs:
    if a in wv and b in wv:
        print(f"{a:10s} ~ {b:10s} cos={cos(wv[a], wv[b]):.3f}")

print("\nAnalogy: king - man + woman ≈ ?")
if all(w in wv for w in ["king","man","woman"]):
    for w,s in wv.most_similar(positive=["king","woman"], negative=["man"], topn=10):
        print(f"  {w:15s} {s:.4f}")


---

## 小结（你现在应当已经“看见了”）

1. **向量与维度**：`|V|`、`D`、具体词的向量；
2. **子采样**：高频词被随机丢弃的概率（你看到了 `p_drop`）；
3. **动态窗口**：本轮用多大窗口、哪些上下文词被当成正例；
4. **负采样**：来自 **unigram$^{0.75}$** 的负例，被用于 `log σ(-⋅)` 这部分；
5. **目标函数**：$\log\sigma(\text{正}) + \sum\log\sigma(-\text{负})$；
6. **参数更新**：两块嵌入矩阵一起被更新，最后导出的是输入嵌入。

如果你愿意，我也可以把 **PyTorch 教学版**改成“训练时每 N 步打印一次：采到的窗口半径、选到的负样本 top-k、当前 loss 与若干词向量的前几维变化”，这样你能**边训边看发生了啥**。
